In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string as str
import pickle

In [ ]:
books = pd.read_csv('Books/Books.csv')
ratings = pd.read_csv('Books/Ratings.csv')
users = pd.read_csv('Books/Users.csv')

In [ ]:
books.head()

In [ ]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]
books.head()

In [ ]:
users.head()

In [ ]:
ratings.head()

In [ ]:
books.isna().sum()

In [ ]:
ratings.isna().sum()

In [ ]:
users.isna().sum()

In [ ]:
books.duplicated().sum()

In [ ]:
ratings.duplicated().sum()

In [ ]:
users.duplicated().sum()

In [ ]:
# user rate more than 200 books
x = ratings['User-ID'].value_counts() > 200

In [ ]:
y = x[x].index
y

In [ ]:
ratings = ratings[ratings['User-ID'].isin(y)]
ratings.head()

In [ ]:
# join ratings with books
ratingsWithBooks = ratings.merge(books, on='ISBN')
ratingsWithBooks.head()

In [ ]:
numberRating = ratingsWithBooks.groupby('Book-Title')['Book-Rating'].count().reset_index()
numberRating.head()

In [ ]:
finalRating = ratingsWithBooks.merge(numberRating, on='Book-Title')
finalRating.head()

In [ ]:
# take books at least 50 rating of user
finalRating = finalRating[finalRating['Book-Rating_y'] >= 50]
finalRating.head()

In [ ]:
finalRating.drop_duplicates(['User-ID', 'Book-Title'], inplace=True)

In [ ]:
# create pivot table
bookPivot = finalRating.pivot_table(columns='User-ID', index='Book-Title', values='Book-Rating_y')
bookPivot.fillna(0, inplace=True)
bookPivot

Training Model

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
bookSparse = csr_matrix(bookPivot)

In [ ]:
# using algo knn
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [ ]:
model.fit(bookSparse)

In [ ]:
distance, suggestion = model.kneighbors(bookPivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )

In [ ]:
print(distance)
print(suggestion)
print(bookPivot.iloc[241,:])

In [ ]:
for i in range(len(suggestion)):
    print(bookPivot.index[suggestion[i]])

In [ ]:
# keep book name
bookNames = bookPivot.index

Find Url

In [ ]:
ids = np.where(finalRating['Book-Title'] == "Harry Potter and the Chamber of Secrets (Book 2)")[0][0]

In [ ]:
finalRating.iloc[ids]['Image-URL-L']

In [ ]:
bookName = []
for bookId in suggestion:
    bookName.append(bookPivot.index[bookId])

In [ ]:
bookName[0]

In [ ]:
idsIndex = []
for name in bookName[0]:
    ids = np.where(finalRating['Book-Title'] == name)[0][0]
    idsIndex.append(ids)

In [ ]:
for idx in idsIndex:
    url = finalRating.iloc[idx]['Image-URL-L']
    print (url)

In [ ]:
finalRating['Storage-Rack'] = np.random.choice(list(str.ascii_lowercase), size=len(finalRating))

In [ ]:
pickle.dump(model, open('model.pkl', 'wb'))
pickle.dump(bookNames, open('bookNames.pkl', 'wb'))
pickle.dump(finalRating, open('finalRating.pkl', 'wb'))
pickle.dump(bookPivot, open('bookPivot.pkl', 'wb'))